# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('data.sqlite')

In [4]:
q = """
SELECT *
FROM offices
"""
pd.read_sql(q, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
q = """
SELECT firstName || " " || lastName AS name
FROM employees
JOIN offices
    ON employees.officeCode = offices.officeCode
WHERE city = 'Boston'
"""
pd.read_sql(q, conn)

,name
0,Julie Firrelli
1,Steve Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
q = """
SELECT offices.officeCode,
       offices.city,
       COUNT(employees.employeeNumber) AS total_employees
FROM employees
RIGHT JOIN offices
    USING (officeCode)
GROUP BY officeCode
HAVING total_employees = 0
"""
pd.read_sql(q, conn)

,officeCode,city,total_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [18]:
"""
Question 1
How many customers are there per office?
"""
q = """
SELECT o.officeCode, o.city, COUNT(c.customerNumber)
FROM offices AS o
JOIN employees AS e
    USING (officeCode)
JOIN customers as c
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode
"""
pd.read_sql(q, conn)

,officeCode,city,COUNT(c.customerNumber)
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [30]:
"""
Question 2
How many product are there per line?
"""

q = """
SELECT l.productLine AS line,
       COUNT(p.productName) AS n_products
FROM products AS p
JOIN productlines AS L
    USING(productLine)
GROUP BY line
"""
pd.read_sql(q, conn)

,line,n_products
0,Classic Cars,38
1,Motorcycles,13
2,Planes,12
3,Ships,9
4,Trains,3
5,Trucks and Buses,11
6,Vintage Cars,24


In [51]:
"""
Question 3
List customer's names, product bought, date ordered and the amount spent
"""

q = """
SELECT c.customerName AS customer,
       pr.productName AS product,
       o.orderDate AS date_ordered,
       pay.amount AS amount_spent
FROM payments AS pay
JOIN customers AS c
    USING(customerNumber)
JOIN orders as o
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products as pr
    USING(productCode)

GROUP BY customer
"""
pd.read_sql(q, conn)

,customer,product,date_ordered,amount_spent
0,"AV Stores, Co.",1965 Aston Martin DB5,2003-03-18,52825.29
1,Alpha Cognac,1993 Mazda RX-7,2003-07-04,33818.34
2,Amica Models & Co.,1952 Alpine Renault 1300,2004-08-17,33924.24
3,"Anna's Decorations, Ltd",1993 Mazda RX-7,2003-09-11,27083.78
4,Atelier graphique,1965 Aston Martin DB5,2003-05-20,6066.78
...,...,...,...,...
93,"Vida Sport, Ltd",1968 Ford Mustang,2004-02-22,47375.92
94,Vitachrome Inc.,1937 Lincoln Berline,2003-01-10,22602.36
95,"Volvo Model Replicas, Co",1952 Alpine Renault 1300,2003-03-24,36005.71
96,West Coast Collectables Co.,1930 Buick Marquette Phaeton,2003-12-01,7678.25


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [56]:
q = """
SELECT e.firstName || " " || e.lastName AS name,
       p.productName AS products
FROM employees AS e
JOIN customers AS c
 ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products as p
    USING(productCode)
;
"""
pd.read_sql(q, conn)

,name,products
0,Leslie Jennings,1958 Setra Bus
1,Leslie Jennings,1940 Ford Pickup Truck
2,Leslie Jennings,1939 Cadillac Limousine
3,Leslie Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie Jennings,1968 Ford Mustang
...,...,...
2991,Martin Gerard,1954 Greyhound Scenicruiser
2992,Martin Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [63]:
q = """
SELECT firstName ,
       lastName,
       SUM(quantityOrdered) AS n_products
FROM employees AS e
JOIN customers AS c
 ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,n_products
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [67]:
q = """
SELECT firstName ||" "||lastName AS name,
       COUNT(productCode) AS diff_products_sold
FROM employees AS e
JOIN customers AS c
 ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING diff_products_sold > 200
ORDER BY diff_products_sold DESC
;
"""
pd.read_sql(q, conn)

,name,diff_products_sold
0,Gerard Hernandez,396
1,Leslie Jennings,331
2,Pamela Castillo,272
3,Larry Bott,236
4,Barry Jones,220
5,George Vanauf,211


In [ ]:
conn.close()

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!